In [1]:
import os
import pandas as pd

current_path = os.getcwd()
print(current_path)
os.chdir('/home/lyq/DataSet/FakeNews/twitter_dataset')
df = pd.read_csv('twitter_llm_rationales.csv')
rationale_names = {'td','cs'}
df[:10]

/home/lyq/PycharmProjects/PycharmProjects


,content,label,image_id,source_id,split,td_rationale,td_pred,td_acc,cs_rationale,cs_pred,cs_acc
0,PIC: Comparison of #Boston suspect Sunil Tripa...,0,boston_fake_23,325145334739267584,NaN,- The message includes a link to an image or ...,0,1,Plausibility: It is plausible that a comparis...,1,0
1,I'm not completely convinced that it's this Su...,0,boston_fake_34,325152091423248385,NaN,The message is a personal opinion or observat...,1,0,Plausibility: The message is a tweet with a l...,0,1
2,4chan and the bombing. just throwing it out th...,0,boston_fake_15,324315545572896768,NaN,1. Source: The source of the message is 4chan...,0,1,Plausibility: The message contains links to t...,0,1
3,4chan thinks they found pictures of the bomber...,0,boston_fake_08,324581777614180352,NaN,The message contains a link to a source (4cha...,0,1,Plausibility: The message contains a link to ...,0,1
4,"Ola ke ase, investigando las bombas de Boston ...",0,boston_fake_35,324665423956176896,NaN,- The message contains a hyperlink to a websi...,0,1,The message is a hashtag and a link to a webs...,0,1
5,@DLoesch have you seen this? Bomber #2 looks ...,0,boston_fake_13,325099014355820544,NaN,- The message contains a link to an image of ...,0,1,Plausibility: It is plausible that a missing ...,1,0
6,"@YourAnonNews its not that guy, you can see hi...",0,boston_fake_16,324558285225209857,NaN,- The message is a Twitter post and does not ...,0,1,Plausibility: It is not believable that the i...,0,1
7,Hilarious if 4chan solves this bombing case. T...,0,boston_fake_26,324435205433724929,NaN,"- The message is a joke about a bombing case,...",0,1,Plausibility: The message is a link to a 4cha...,0,1
8,http://t.co/pS4KetWGaW #4chan,0,boston_fake_01,324545148908363777,NaN,1. The message contains a link to a website c...,0,1,Plausibility: The message is a link to a webs...,0,1
9,"\""They say\"" this is a #pic of a man lurking o...",0,boston_fake_09,324267996044460035,NaN,"- The message includes a hashtag and a URL, w...",0,1,Plausibility: The idea of a man lurking on a ...,0,1


In [2]:
print(f"sum: {df.shape[0]}")
value_count = df['label'].value_counts()
print(f"real data: {value_count.get(1,0)}")
print(f"fake data: {value_count.get(0,0)}")

sum: 14195
real data: 6432
fake data: 7763


In [3]:
def filter_data(df):
    for r_name in rationale_names:
        df = df[ (df[f'{r_name}_pred'] != -1) & (df[f'{r_name}_rationale'] is not None) ]
    return df
        
print(f"before filtering : {df.shape[0]}  real data :{(df['label']==1).sum()} fake data: {(df['label']==0).sum()}")
df = filter_data(df)
print(f"after filtering : {df.shape[0]}  real data :{(df['label']==1).sum()} fake data: {(df['label']==0).sum()}")


before filtering : 14195  real data :6432 fake data: 7763
after filtering : 14195  real data :6432 fake data: 7763


In [4]:
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix, accuracy_score

for r_name in rationale_names:
    pred = df[r_name+ "_pred"].tolist()
    label = df['label'].tolist()
    print(f"{r_name}: acc {accuracy_score(label, pred)} recall_real: {recall_score(label, pred)} recall_fake: {recall_score(label, pred,pos_label=0)} precision_real: {precision_score(label, pred)} precision_fake: {precision_score(label, pred,pos_label=0)}")
    

td: acc 0.578513561113068 recall_real: 0.3787313432835821 recall_fake: 0.7440422517068144 precision_real: 0.5507574044765996 precision_fake: 0.5910765452312731
cs: acc 0.5692145121521662 recall_real: 0.4528917910447761 recall_fake: 0.6655931985057323 precision_real: 0.5287711018333636 precision_fake: 0.5948653004835367


In [5]:
def split_data0(df,train_ratio=0.8,valid_ratio=0.1):
    train_nums = int(df.shape[0]*train_ratio)
    valid_nums = int(df.shape[0]*valid_ratio)
    return df.iloc[:train_nums],df.iloc[train_nums:train_nums+valid_nums],df.iloc[train_nums+valid_nums:]

def split_data(df,train_ratio=0.8,valid_ratio=0.1):
    real_data = df[df['label'] == 1]
    fake_data = df[df['label'] == 0]
    print(f"real_data count : {real_data.shape[0]}")
    print(f"fake_data count : {fake_data.shape[0]}")
    train_real,valid_real,test_real = split_data0(real_data,train_ratio,valid_ratio)
    train_fake,valid_fake,test_fake = split_data0(fake_data,train_ratio,valid_ratio)
    train_df,valid_df,test_df = pd.concat([train_real,train_fake]),pd.concat([valid_real,valid_fake]),pd.concat([test_real,test_fake])
    train_df['split'] = 'train'
    valid_df['split'] = 'valid'
    test_df['split'] = 'test'
    return train_df,valid_df,test_df


train_df,valid_df,test_df = split_data(df,train_ratio=0.8,valid_ratio=0.1)


train_df.shape[0],valid_df.shape[0],test_df.shape[0]

real_data count : 6432
fake_data count : 7763


(11355, 1419, 1421)

In [6]:
train_df.to_csv('train.csv',index=False)
valid_df.to_csv('val.csv',index=False)
test_df.to_csv('test.csv',index=False)